In [1]:
from Acesso import Login
from Query import Query
from RemoverArquivo import Remover
from Moeda import Moeda
from Tempo import DataHora
from glob import glob
import os
import pandas as pd

pd.set_option('display.max_columns',None)

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

data=DataHora()

querys={

    'Roteiro':

    """

    SELECT * FROM netfeira.vw_roteira_visita
    WHERE Categoria='CLT'
    
    """,

    'Calendario':

    """
    
    DECLARE @DTInicial AS DATETIME,@DTFinal AS DATETIME

    SET @DTInicial='2018-01-01'
    SET @DTFinal= CONCAT(YEAR(GETDATE())+1,'-01-','01')

    ;WITH Calendario (Datas) AS(

    SELECT @DTInicial
    UNION ALL
    SELECT Datas+1
    FROM Calendario WHERE  Datas+1<@DTFinal
    )

    SELECT CONVERT(DATETIME,CAST(Datas AS DATE),101) AS 'Data',YEAR(Datas) AS 'Ano',MONTH(Datas) AS 'Cód. Mês',
    CHOOSE(MONTH(Datas),'JANEIRO','FEVEREIRO','MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO','DEZEMBRO') AS 'Mês',
    CHOOSE(MONTH(Datas),'JAN','FEV','MAR','ABR','MAI','JUN','JUL','AGO','SET','OUT','NOV','DEZ') AS 'Mês Resumido',
    DAY(Datas) AS 'Dia',CONVERT(VARCHAR(7),Datas,120) AS 'Mês Meta',
    DATEPART(DW,Datas) AS 'Cód. Semana',CHOOSE(DATEPART(DW,Datas),'DOM','SEG','TER','QUAR','QUI','SEX','SÁB') AS 'Semana',
    CASE WHEN DATEPART(DW,Datas) IN (7,1) THEN 0 ELSE 1 END AS 'Dias Úteis',
    CASE WHEN MONTH(Datas)<=3 THEN '1º TRIM' WHEN MONTH(Datas)<=6 THEN '2º TRIM' WHEN MONTH(Datas)<=9 THEN '3º TRIM' WHEN MONTH(Datas)<=12 THEN '4º TRIM' END AS 'Trimestre Ano',
    CASE WHEN MONTH(Datas)<=6 THEN '1º SEM' ELSE '2º SEM' END AS 'Semestre Ano',DATEPART(WEEK,Datas) AS 'Semana Ano',
    CONVERT(VARCHAR,DAY(Datas))+'/'+CONVERT(VARCHAR,MONTH(Datas)) AS 'Mês Base'
    FROM Calendario OPTION(MAXRECURSION 10000)    
        
    """

}

In [2]:
tabela_df=sql.CriarTabela(kwargs=querys)

In [3]:
tabela_df['Calendario'].columns

Index(['Data', 'Ano', 'Cód. Mês', 'Mês', 'Mês Resumido', 'Dia', 'Mês Meta',
       'Cód. Semana', 'Semana', 'Dias Úteis', 'Trimestre Ano', 'Semestre Ano',
       'Semana Ano', 'Mês Base'],
      dtype='object')

In [4]:
tabela_df['Roteiro'].columns

Index(['ID Semana', 'Roteiro Semana', 'Seq', 'ID Cliente', 'Razão Social',
       'Nome Fantasia', 'Status do Cliente', 'Segmento', 'Canal', 'Matriz',
       'Endereço', 'Bairro', 'Cidade', 'UF', 'Numero', 'DDD', 'Contato',
       'ID Vendedor', 'Nome', 'Nome Resumido', 'Equipe', 'E-mail', 'Categoria',
       'Supervisor', 'Email Sup', 'Gerente', 'Email Gerente'],
      dtype='object')

In [5]:
tabela_df['Calendario']=tabela_df['Calendario'].merge(tabela_df['Roteiro'],left_on='Cód. Semana',right_on='ID Semana',how='inner')[['Data', 'Ano', 'Cód. Mês', 'Mês', 'Mês Resumido', 'Dia','Roteiro Semana', 'Seq', 'ID Cliente', 'Razão Social',
       'Nome Fantasia', 'Status do Cliente', 'Segmento', 'Canal', 'Matriz',
       'Endereço', 'Bairro', 'Cidade', 'UF', 'Numero', 'DDD', 'Contato',
       'ID Vendedor', 'Nome', 'Nome Resumido', 'Equipe', 'E-mail', 'Categoria',
       'Supervisor', 'Email Sup', 'Gerente', 'Email Gerente']]

In [6]:
data_atual=data.HoraAtual()

data_atual

datetime.datetime(2022, 8, 19, 17, 55, 5)

In [7]:
tabela_df['Calendario'].loc[(tabela_df['Calendario']['Data'].dt.year==data_atual.year)&(tabela_df['Calendario']['Data'].dt.month==data_atual.month)&(tabela_df['Calendario']['Data'].dt.day==data_atual.day)]

,Data,Ano,Cód. Mês,Mês,Mês Resumido,Dia,Roteiro Semana,Seq,ID Cliente,Razão Social,Nome Fantasia,Status do Cliente,Segmento,Canal,Matriz,Endereço,Bairro,Cidade,UF,Numero,DDD,Contato,ID Vendedor,Nome,Nome Resumido,Equipe,E-mail,Categoria,Supervisor,Email Sup,Gerente,Email Gerente
221200,2022-08-19,2022,8,AGOSTO,AGO,19,SEXTA-FEIRA,9164,16930,COMERCIAL DE ALIMENTOS RED LTDA,COMERCIAL DE ALIMENT,ATIVO,1 A 4 CHK,AS,COMERCIAL DE ALIMENT,AV DOUTOR CANDIDO MOTTA FILHO,CIDADE SÃO FRANCISCO,SÃO PAULO,SP,338,11,9.829706e+08,CLAUDIOP,CLAUDIO TADEU PEREIRA,CLAUDIO PEREIRA,EQUIPE 3 (CLT),DEMARCHI.CLAUDIOP@GMAIL.COM,CLT,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR
221201,2022-08-19,2022,8,AGOSTO,AGO,19,SEXTA-FEIRA,9163,17108,SUPERMERCADO BRUNO LTDA,SUPERMERCADO BRUNO,ATIVO,1 A 4 CHK,AS,SUPERMERCADO BRUNO,AV PLUTAO,JARDIM NOVO HORIZONTE,CARAPICUÍBA,SP,583,11,4.186077e+07,CLAUDIOP,CLAUDIO TADEU PEREIRA,CLAUDIO PEREIRA,EQUIPE 3 (CLT),DEMARCHI.CLAUDIOP@GMAIL.COM,CLT,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR
221202,2022-08-19,2022,8,AGOSTO,AGO,19,SEXTA-FEIRA,9149,17173,NICOLAU CARLOS ETEROVIC E CIA LTDA EPP,ARMAZEM DO NICOLAU,ATIVO,1 A 4 CHK,AS,None,RUA OURO PRETO,CHÁCARA DAS PAINEIRAS,CARAPICUÍBA,SP,561,0,1.111111e+10,CLAUDIOP,CLAUDIO TADEU PEREIRA,CLAUDIO PEREIRA,EQUIPE 3 (CLT),DEMARCHI.CLAUDIOP@GMAIL.COM,CLT,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR
221203,2022-08-19,2022,8,AGOSTO,AGO,19,SEXTA-FEIRA,9150,19231,SUPERMERCADO PINHEIRO & BOLIVAR LTDA,SUPERMERCADO PINHEIR,ATIVO,1 A 4 CHK,AS,None,ESTRADA DOUTOR MIGUEL VIEIRA FERREIRA,JARDIM PLANALTO,CARAPICUÍBA,SP,620,11,4.187205e+07,CLAUDIOP,CLAUDIO TADEU PEREIRA,CLAUDIO PEREIRA,EQUIPE 3 (CLT),DEMARCHI.CLAUDIOP@GMAIL.COM,CLT,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR
221204,2022-08-19,2022,8,AGOSTO,AGO,19,SEXTA-FEIRA,9160,19686,SUPERMERCADO BRUNO LTDA,SUPERMERCADO BRUNO,ATIVO,1 A 4 CHK,AS,SUPERMERCADO BRUNO,RUA ITAJUBA,PARQUE SANTA TERESA,CARAPICUÍBA,SP,363,11,4.146935e+07,CLAUDIOP,CLAUDIO TADEU PEREIRA,CLAUDIO PEREIRA,EQUIPE 3 (CLT),DEMARCHI.CLAUDIOP@GMAIL.COM,CLT,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221358,2022-08-19,2022,8,AGOSTO,AGO,19,SEXTA-FEIRA,8691,21853,BAZE BAR E RESTAURANTE LTDA,BAZE BAR,ATIVO,RESTAURANTE,FS,None,AVENIDA PRESIDENTE JUSCELINO KUBITSCHEK - LADO...,VILA NOVA CONCEIÇÃO,SÃO PAULO,SP,1082,11,9.930013e+08,MAYRAQS,MAYRA QUEIROS DOS SANTOS,MAYRA SANTOS,EQUIPE 9 (ATIVO),MAYRA.SANTOS@DEMARCHISAOPAULO.COM.BR,CLT,RAQUEL SOBRINHO,RAQUEL@DEMARCHISAOPAULO.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR
221359,2022-08-19,2022,8,AGOSTO,AGO,19,SEXTA-FEIRA,8110,8610,SORAYA ELIE AKIAMA KASHIHARA,SORAYA ELIE AKIAMA K,ATIVO,RESTAURANTE,FS,None,"RUA DOUTOR BARROS CRUZ, 65",VILA MARIANA,SÃO PAULO,SP,65,11,5.081317e+07,RAQUELSO,RAQUEL SOBRINHO,RAQUEL SOBRINHO,EQUIPE 9 (ATIVO),RAQUEL@DEMARCHISAOPAULO.COM.BR,CLT,RAQUEL SOBRINHO,RAQUEL@DEMARCHISAOPAULO.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR
221360,2022-08-19,2022,8,AGOSTO,AGO,19,SEXTA-FEIRA,8219,8676,DINAMITE ITAIM CHOPERIA LTDA,BOTECO SAO BENTO ITA,ATIVO,BAR/LANCHONETE,FS,None,RUA JOAO CACHOEIRA,VILA NOVA CONCEIÇÃO,SÃO PAULO,SP,800,11,5.052573e+07,RAQUELSO,RAQUEL SOBRINHO,RAQUEL SOBRINHO,EQUIPE 9 (ATIVO),RAQUEL@DEMARCHISAOPAULO.COM.BR,CLT,RAQUEL SOBRINHO,RAQUEL@DEMARCHISAOPAULO.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR
221361,2022-08-19,2022,8,AGOSTO,AGO,19,SEXTA-FEIRA,8215,19840,GMO CAFE COMERCIO DE ALIMENTOS LTDA EP,GMO CAFE,ATIVO,CAFETERIA,FS,None,"PRAÇA DA LIBERDADE, 169",LIBERDADE,SÃO PAULO,SP,169,11,9.811011e+08,RAQUELSO,RAQUEL SOBRINHO,RAQUEL SOBRINHO,EQUIPE 9 (ATIVO),RAQUEL@DEMARCHI